### 1. Two type of programs
In the program word, there are two tipical program types:
- **IO-bound** : this type program spend most of their time waiting for input/output. 
- **CPU bound** : this type program spend most of their time performing calculations in CPU.

Let's have a look at a **IO-bound** type program example:


In [2]:

import requests
from concurrent.futures import ThreadPoolExecutor

urls = [
  'http://www.python.org',
  'https://docs.python.org/3/',
  'https://docs.python.org/3/whatsnew/3.7.html',
  'https://docs.python.org/3/tutorial/index.html',
  'https://docs.python.org/3/library/index.html',
  'https://docs.python.org/3/reference/index.html',
  'https://docs.python.org/3/using/index.html',
  'https://docs.python.org/3/howto/index.html',
  'https://docs.python.org/3/installing/index.html',
  'https://docs.python.org/3/distributing/index.html',
  'https://docs.python.org/3/extending/index.html',
  'https://docs.python.org/3/c-api/index.html',
  'https://docs.python.org/3/faq/index.html'
  ]

In [14]:
%%time

results = []
for url in urls:
    with requests.get(url) as response:
        results.append(response.content)

CPU times: user 182 ms, sys: 13.2 ms, total: 195 ms
Wall time: 767 ms


This program simply fetch data from a few URLs. The CPU time is only 195 ms, but the total time is 767 ms.

Now, let's have a look at an example **CPU bound** type program:

In [16]:
%%time
def is_prime(n):
    if n <= 3 and n > 1:
        return n
    elif n % 2 == 0 or n % 3 == 0:
        return 0

    i = 5

    while i * i <= n :
        if n % i == 0 or n % (i + 2) == 0:
            return 0
        i = i + 6

    return n

answer = 0
n = 100000
for i in range(n):
    answer += is_prime(i)

CPU times: user 128 ms, sys: 4.42 ms, total: 132 ms
Wall time: 138 ms


We can find out that the `132/138 = 95.65%` execution time has been consumed in calculation.
### 2. Why do we need mutil-thread and multi-process in python
The reason is quite simple, to make a better usage of cpu source, or speed up your program.
Let's add multi-thread to the **IO-bound** example:


In [10]:
%%time
with ThreadPoolExecutor(2) as executor:
    results = executor.map(requests.get, urls)

CPU times: user 181 ms, sys: 15 ms, total: 196 ms
Wall time: 428 ms


In [12]:
%%time
with ThreadPoolExecutor(4) as executor:
    results = executor.map(requests.get, urls)

CPU times: user 194 ms, sys: 18.1 ms, total: 212 ms
Wall time: 290 ms


In [13]:
%%time
with ThreadPoolExecutor(6) as executor:
    results = executor.map(requests.get, urls)

CPU times: user 215 ms, sys: 19.8 ms, total: 235 ms
Wall time: 221 ms


Results:

|            |  1 thread  |  2 threads  |  4 threads  |  6 threads  |
|:----------:|:----------:|:-----------:|:-----------:|:-----------:|
|  CPU time  |   195 ms   |    196 ms   |    212 ms   |    235 ms   |
| total time |   767 ms   |    428 ms   |    290 ms   |    221 ms   |

It is clear that the CPU time increased and wall time decreases.

We can also imporve performance if we apply multi-process for **IO-bound** program, but the overhead tends to be higher than using multi-thread. 

Now let's apply multi-thread to **CPU bound** program:

In [33]:
%%time
answer = 0
with ThreadPoolExecutor(2) as executor:
    answer = sum(executor.map(is_prime, list(range(n)), chunksize=(n/2)))

CPU times: user 2.48 s, sys: 111 ms, total: 2.6 s
Wall time: 2.68 s


In [47]:
%%time
answer = 0
with ThreadPoolExecutor(4) as executor:
    answer = sum(executor.map(is_prime, list(range(n)), chunksize=(n/4)))

CPU times: user 2.62 s, sys: 109 ms, total: 2.73 s
Wall time: 2.81 s


In [46]:
%%time
answer = 0
with ThreadPoolExecutor(8) as executor:
    answer = sum(executor.map(is_prime, list(range(n)),chunksize=(n/8)))

CPU times: user 2.76 s, sys: 102 ms, total: 2.86 s
Wall time: 2.91 s


The result table is:

|            |  2 thread  |  4 threads  |  8 threads  | 
|:----------:|:----------:|:-----------:|:-----------:|
|  CPU time  |   2.6 s    |    2.73 s   |    2.86 s   |
| total time |   2.68 s   |    2.81 s   |    2.91 s   |
It is very strange that when we add more threads we get worse result.

Actually, there are a few different implementation of python interpreter:
- **CPython**: This interpreter the official interpreter written by C language with a Global Interpreter Lock (GIL). GIL is a mechanism used in computer-language interpreters to synchronize the execution of threads so that only one native thread can execute at a time.
- **PyPy**: This interpreter itself is written in a restricted subset of Python called RPython (Restricted Python).RPython puts some constraints on the Python language such that a variable's type can be inferred at compile time. PyPy often runs faster than CPython because PyPy is a just-in-time compiler while CPython is an interpreter.
- **Jython**: It is an implementation of the Python programming language designed to run on the Java platform. Jython programs can import and use any Java class. Except for some standard modules, Jython programs use Java classes instead of Python modules. Jython includes almost all of the modules in the standard Python programming language distribution, lacking only some of the modules implemented originally in C.

Usually, in this example, we used the official implementation, the CPython. Because of GIL, only one thread will be executed at any time. The more threads we add to thread pool, the more time it will consume when scheduling threads.

How about mutil-process?



In [55]:
from multiprocessing import Pool
def is_prime(n):
    if n <= 3 and n > 1:
        return n
    elif n % 2 == 0 or n % 3 == 0:
        return 0

    i = 5

    while i * i <= n :
        if n % i == 0 or n % (i + 2) == 0:
            return 0
        i = i + 6

    return n
n = 1000000

In [60]:
%%time

if __name__ == '__main__':
    with Pool(1) as p:
        answer = sum(p.map(is_prime, list(range(n))))

CPU times: user 119 ms, sys: 64.7 ms, total: 183 ms
Wall time: 2.69 s


In [61]:
%%time

if __name__ == '__main__':
    with Pool(2) as p:
        answer = sum(p.map(is_prime, list(range(n))))

CPU times: user 116 ms, sys: 63.1 ms, total: 179 ms
Wall time: 1.57 s


In [62]:
%%time
if __name__ == '__main__':
    with Pool(4) as p:
        answer = sum(p.map(is_prime, list(range(n))))

CPU times: user 126 ms, sys: 60.1 ms, total: 186 ms
Wall time: 957 ms


In [63]:
%%time
if __name__ == '__main__':
    with Pool(8) as p:
        answer = sum(p.map(is_prime, list(range(n))))

CPU times: user 143 ms, sys: 72.7 ms, total: 216 ms
Wall time: 875 ms


In [64]:
%%time
if __name__ == '__main__':
    with Pool(32) as p:
        answer = sum(p.map(is_prime, list(range(n))))

CPU times: user 185 ms, sys: 142 ms, total: 328 ms
Wall time: 1.01 s


|            |  1 processes  |  2 processes  |  4 processes  |  8 processes  | 32 processes  |
|:----------:|:----------:|:-----------:|:-----------:|:-----------:|:--:|
|  CPU time  |   183 ms   |    179 ms   |    186 ms   |    216 ms   |328 ms|
| total time |   2.69 s   |    1.57 s   |    957 ms   |    875 ms   |1.01 s|

That's it! The more processes we have, the faster result we can get. But please note that one process usually will occupy one core of CPU. Your process number should be equal to or less than the CPU cores, otherwise Schedule extra process will comsume more CPU resource and eventually slow your program.

### 3. TLDR
The conclusion is based on the offical python interperter CPython.
- **IO-bound** program should use multi-thread to improve performance. When you feel like one core is not enough for your program, you can combine mutil-thread and mutil-process.
- **CPU bound** program can use multi-process to improve performance. But limit your processes number based on your situation.
